In [7]:

#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sat Jun 12 2021
@author: cechava
"""
from itertools import groupby
import numpy as np
import pandas as pd
import scipy
import scipy.signal

#to visualize 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")


#to load files
import os
import h5py

#ML packages
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import f1_score,make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE


from tensorflow import keras
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential, Model, load_model, Sequential, save_model
from tensorflow. keras.layers import Dense, Activation, Dropout, Input,  TimeDistributed, GRU, Masking, LSTM
from tensorflow.keras.utils import to_categorical

from utils import *

In [93]:
def get_RNN_f1(X, Y, model, average = 'weighted', mask_value = -100):
    """
    Get f1 score for an RNN model using masked timepoint data

    Args:
        X: 3D numpy array with shape [samples, timepoints, features]
        Y: 3D numpy array with shape [samples, timepoints, classes]. one-hot coding of classes
        model: RNN model object
        average: string argument for f1_score function. Usually 'macro' or 'weighted'
        mask_value: value indicating which timepoints to mask out

    Returns:
        f1: f1 score
    """
    # Mask out indices based on mask value
    nonmasked_idxs = np.where(X[:,:,0].flatten()!=mask_value)[0]
    # Get target labels for non-masked timepoints
    y_true = np.argmax(Y,2).flatten()[nonmasked_idxs]
    # Get model predictions for non-masked timepoints
    preds = model.predict(X)
    y_pred = np.argmax(preds,2).flatten()[nonmasked_idxs]
    # Get F1 score
    f1 = f1_score(y_true,y_pred,average = average)

    return f1

In [120]:
#define where the data file are located
data_folder = './EMG_data/01'


#filter parameters
lo_freq = 20
hi_freq = 450

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case I want to re-run later with some overlap

#get features across segments and corresponding info
feature_matrix, target_labels, window_tstamps, \
block_labels, series_labels = get_subject_data_for_classification(data_folder, lo_freq, hi_freq, \
                                                                  win_size, step)

#exclude blocks with 'unknown' label
in_samples = np.where(target_labels != 0)[0]
feature_matrix_in = feature_matrix[in_samples,:]
target_labels_in = target_labels[in_samples]
window_tstamps_in = window_tstamps[in_samples]
block_labels_in = block_labels[in_samples]


feature_matrix = feature_matrix_in.T
target_labels = target_labels_in
window_tstamps_ = window_tstamps_in
block_labels = block_labels_in



In [129]:
n_splits = 4
verbose = 1
epochs = 10
batch_size = 2
permute = True

#empty arrays
train_f1_scores = np.empty((n_splits,))
test_f1_scores = np.empty((n_splits,))

#get block_ids and corresponding classes in block. there are the units over which we will do train/test split
blocks = np.array([k for k,g in groupby(block_labels) if k!=0])
classes = np.array([k for k,g in groupby(target_labels) if k!=0])

#permute class labels, if indicated
if permute:
    #using indexing tricks to have this work out
    classes_perm = np.random.permutation(classes)
    target_labels_shuffled = np.empty((0,))
    for i,b in enumerate(blocks):
        idxs = np.where(block_labels==b)[0]
        target_labels_shuffled = np.hstack((target_labels_shuffled,classes_perm[i]*np.ones((idxs.size,))))
    target_labels = target_labels_shuffled
    classes = classes_perm


#stratify split to retain ratio of class labels
skf = StratifiedKFold(n_splits=n_splits,shuffle = True)

#systematically use one fold of the data as a held-out test set
for split_count, (blocks_train_idxs, blocks_test_idxs) in enumerate(skf.split(blocks, classes)):
    print('Split Count: %i'% (split_count+1))

    #get train and test indices
    blocks_train = blocks[blocks_train_idxs]
    blocks_test = blocks[blocks_test_idxs]
    train_idxs =np.where(np.isin(block_labels,blocks_train))[0]
    test_idxs =np.where(np.isin(block_labels,blocks_test))[0]

    # select training data and pad to get an array where each sample has same number of timesteps
    X_train = feature_matrix[:,train_idxs]
    y_train = target_labels[train_idxs]
    #one-hot encoding of class labels
    y_train = to_categorical(y_train-np.min(y_train))
    #get block labels of given samples
    win_blocks_train = block_labels[train_idxs]

    #get cube
    X_train_cube, Y_train_cube, scaler = get_data_cube(X_train, y_train,win_blocks_train, train = True, magic_value = -100)
    print(X_train_cube.shape, Y_train_cube.shape)

    # select test data and pad to get an array where each sample has same number of timesteps
    X_test = feature_matrix[:,test_idxs]
    y_test = target_labels[test_idxs]
    #one-hot encoding of class labels
    y_test = to_categorical(y_test-np.min(y_test))
    #get block labels of given samples
    win_blocks_test = block_labels[test_idxs]
    #get data cube
    X_test_cube, Y_test_cube, scaler = get_data_cube(X_test, y_test, win_blocks_test, train = False, scaler = scaler, magic_value = -100)
    print(X_test_cube.shape, Y_test_cube.shape)

    n_timesteps, n_features, n_outputs = X_train_cube.shape[1], X_train_cube.shape[2], Y_test_cube.shape[2]

    #setting timestep dimension to None for variable length input
    model = many_to_many_model((None,n_features),n_outputs,mask_value = -100)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary

    #fit model
    model.fit(X_train_cube, Y_train_cube, epochs=epochs, batch_size=batch_size, verbose=verbose)

    #evaluate model
    train_f1_scores[split_count] = get_RNN_f1(X_train_cube, Y_train_cube, model)
    test_f1_scores[split_count] = get_RNN_f1(X_test_cube, Y_test_cube, model)

Split Count: 1
(18, 21, 16) (18, 21, 6)
(6, 20, 16) (6, 20, 6)
Train on 18 samples
Epoch 1/10
18/18 [==============================] - 4s 240ms/sample - loss: 1.8063 - acc: 0.0867
Epoch 2/10
18/18 [==============================] - 1s 28ms/sample - loss: 1.7323 - acc: 0.1579
Epoch 3/10
18/18 [==============================] - 0s 23ms/sample - loss: 1.6863 - acc: 0.1548
Epoch 4/10
18/18 [==============================] - 0s 23ms/sample - loss: 1.6688 - acc: 0.1455
Epoch 5/10
18/18 [==============================] - 0s 24ms/sample - loss: 1.6041 - acc: 0.1920
Epoch 6/10
18/18 [==============================] - 0s 25ms/sample - loss: 1.5576 - acc: 0.2477
Epoch 7/10
18/18 [==============================] - 0s 24ms/sample - loss: 1.5578 - acc: 0.2105
Epoch 8/10
18/18 [==============================] - 0s 27ms/sample - loss: 1.4720 - acc: 0.2910
Epoch 9/10
18/18 [==============================] - 0s 27ms/sample - loss: 1.5015 - acc: 0.3003
Epoch 10/10
18/18 [==============================] -

In [130]:
train_f1_scores

array([0.3002669 , 0.36841417, 0.28775529, 0.43316632])

In [131]:
test_f1_scores

array([0.02708995, 0.17571759, 0.18739084, 0.00801887])

In [127]:
train_f1_scores

array([0.84589639, 0.63602198, 0.76269969, 0.59607104])

In [128]:
test_f1_scores

array([0.95247002, 0.51858536, 0.57616385, 0.60577765])